In [1]:
import numpy as np

# add parent directory to path
import os, sys
sys.path.append('..')

from utils.mmap_dataset import MMapIndexedDataset
import dask
import dask.array as da
from transformers import GPTNeoXForCausalLM, AutoTokenizer
import matplotlib.pyplot as plt
from dask.diagnostics import ProgressBar

In [2]:
x0 = np.random.randint(0, 10, size=(30000, 100))
x1 = x0[:10, :5]
def match(a, b):
    matches = np.empty([a.shape[0], b.shape[0]], dtype=bool)
    matches.fill(False)
    # return np.dot(a,b.T).reshape(1, 1, 1, 1)
    for i in range(b.shape[1] - a.shape[1]):
        matches = np.logical_or(matches,
                                np.max(np.abs(np.expand_dims(a, axis=1) - b[:, i:i+a.shape[1]]), axis=-1) == 0)
    return np.expand_dims(np.expand_dims(matches, -1), -1)
x0 = da.from_array(x0, chunks=(500, 100))
x1 = da.from_array(x1, chunks=(5, 5))
res = da.blockwise(match, 'ijab', x1, 'ia', x0, 'jb', dtype=int, 
                   adjust_chunks={'a': 1, 'b': 1})

In [3]:
ProgressBar().register()

In [4]:
result = res.compute()

[                                        ] | 0% Completed | 331.10 us

[########################################] | 100% Completed | 1.80 sms


In [6]:
result.squeeze().shape

(10, 30000)

In [2]:

dataset = MMapIndexedDataset('/om/user/sunnyd/data/datasets--EleutherAI--pile-standard-pythia-preshuffled-merged/document', skip_warmup = True)

    reading sizes...
    reading pointers...
    reading document index...
    creating numpy buffer of mmap...
    creating memory view of numpy buffer...


In [4]:
def load_chunk(idx, blocksize):
    return dataset[idx*blocksize:(idx+1)*blocksize]

def mmap_dask_array(blocksize=1000):
    load = dask.delayed(load_chunk)
    chunks = []
    for index in range(0, len(dataset), blocksize):
        # Truncate the last chunk if necessary
        chunk_size = min(blocksize, len(dataset) - index)
        chunk = dask.array.from_delayed(
            load(index, chunk_size),
            shape=(chunk_size, 2049),
            dtype=dataset[0].dtype
        )
        chunks.append(chunk)
    return da.concatenate(chunks, axis=0)

x = mmap_dask_array(100000)

In [41]:
x

dask.array<concatenate, shape=(146432000, 2049), dtype=uint16, chunksize=(100000, 2049), chunktype=numpy.ndarray>

In [15]:
x0 = x[:10]

x0 = np.random.randn(10, 100)

def gufoo(x, y):
    for j in range(y.shape[0]):
        res[j] = 0
        for i in range(y.shape[1] - x.shape[0]):
            if np.max(np.abs(x[i:i+x.shape[0]] - y[:, i:i+x.shape[0]])) > 0:
                res[j] = 1


y = gufoo(x0[0, :5], x0)
y.compute()

TypeError: gufoo() missing 1 required positional argument: 'res'

In [145]:
x1

dask.array<array, shape=(20, 10), dtype=int64, chunksize=(1, 10), chunktype=numpy.ndarray>

In [142]:
# x
# da.map_blocks(
#     match,
#     x0,
#     x1,
#     dtype=bool
# ).compute()

/tmp/ipykernel_3074442/2057689032.py:1: PerformanceWarning: Increasing number of chunks by factor of 20
  da.blockwise(match, 'ij', x0, 'ia', x1, 'jb', dtype=bool).compute()


AttributeError: 'list' object has no attribute 'reshape'

In [130]:
x = da.from_array([[1, 2],
                   [3, 4]], chunks=(1, 2))
y = da.from_array([[10, 20],
                   [0, 0]])
z = da.blockwise(lambda x, y: x + y.shape[0], 'ij', x, 'ij', y, 'ji', dtype='f8')
z.compute()

array([[3, 4],
       [5, 6]])

In [112]:
import operator
operator.mod([], [])

TypeError: unsupported operand type(s) for %: 'list' and 'list'

In [124]:
a = da.from_array([0, 1, 2], chunks=1)
b = da.from_array([10, 50, 100], chunks=1)
def outer1(a, b):
    print(type(a), b.shape)
    return (a - b).reshape(1, 1)
    # print(a, b)
    # print(np.outer(a, b))
    # return np.outer(a, b)
    if a.size ==  0:
       return np.outer(a, b)
    else:
        print("hi")
    print(np.product(a, b))
    return np.product(a, b)
z = da.blockwise(outer1, 'ij', a, 'i', b, 'j', dtype='f8')
z.compute()

<class 'numpy.ndarray'> (0,)


array([[ -10,  -50, -100],
       [  -9,  -49,  -99],
       [  -8,  -48,  -98]])

In [28]:
import dask.array as da

def add_arrays(a, b, out):
  # Perform element-wise addition
  out[:] = a + b

add_gufunc = da.gufunc(add_arrays, signature="(m,n),(m,n)->(m,n)", output_dtypes=float)
array1 = da.random.random(5, 5)
array2 = da.random.random(5, 5)
print(array1.dtype)

# Execute the gufunc
result = add_gufunc(array1, array2)

# Calculate the result asynchronously (optional)
result = result.compute()

float64


ValueError: axes don't match array

In [17]:
len(dataset)

146432000

## TODO:
1. Find a way to convert back to natural text
2. Load data into dask


In [19]:
tokenizer = AutoTokenizer.from_pretrained(
  "EleutherAI/pythia-70m-deduped",
  revision="step3000",
  cache_dir="/om/user/sunnyd/transformers_cache",
)

inputs = tokenizer("Hello, I am", return_tensors="pt")
# tokens = model.generate(**inputs)
# tokenizer.decode(tokens[0])

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
tokenizer.decode(dataset[15])

'\nThe rocking Südtribüne, a charismatic coach with his shoestring team, spearheaded by their cherubic talisman. Borussia Dortmund had well and truly shaken Bavaria, in a time where oligarchs and billionaire investors are dictating the<|endoftext|>Kim Cheong-gi\n\nKim Cheong-gi (born April 4, 1941) is a South Korean director of animated, fantasy, and science fiction films.\n\nHis work, Space Gundam V (1983), is considered one of the milestones of Korean animation of the 1980s. With actor Shim Hyung-rae, he created the Ureme series, one of the more popular Korean children\'s series of the late 1980s.\n\nPartial filmography \nRobot Taekwon V (animated) (1976)\nRobot Taekwon V 3 (로보트 태권 V 3탄: 수중특공대) (animated) (1977)\nGolden Wing 123 (황금날개 123) (animated) (1978)\nRun, Wonder Princess! (날아라 원더공주) (1978)\nTale of Three Kingdoms (삼국지) (animated) (1980)\nSuper Taekwon V (수퍼 태권브이) (animated) (1982)\nWuroemae from the Outside (우뢰매 = \'Wuroemae,\' or \'Ureme,\' \'Uremae,\' \'Uroi-mae,\' etc.) (외